Как автор `recordclass` хотелось бы пролить немного света.

Recorclass возник как ответ на [вопрос](https://stackoverflow.com/questions/29290359/existence-of-mutable-named-tuple-in-python/29419745#29419745) на stackoverflow. 

`Recordclass` задумывался и реализовывался как тип, который по api, памяти и скорости был бы полностью идентичен `namedtuple` за исключением того, что он поддерживал бы присваивание при котором можно было заменить любой элемент, не создавая при этом новый экземпляр, как в `namedtuple`. Т.е. был бы почти полностью идентичен `namedtuple` и поддерживал присваивание (`__setitem__`/`setslice__`).

Эффективность namedtuple бизируется на эффективности типа `tuple` в python. Для того, чтобы добиться такой же эффективности пришлось создать тип `memoryslots`. Его структура (`PyMemorySlotsObject`) идентична структуре `tuple` (`PyTupleObject`) и поэтому занимает такой же объем памяти, что и `tuple`.
`memoryslots` реализует только `__getitem__`/`__setitem__`, `__getslice__`/`__setslice__`, `__len__`, `__iter__`, `__cmp__`, `__richcmp__`. Этого достаточно для нужд реализации.

`Recordclass` определяется поверх `memoryslots` точно так же как и `namedtuple` поверх `tuple`. Доступ к атрибутам осуществляется посредством дескриптора (`itemgetset`), который предоставляет быстрый доступ и присваивание по индексу атрибута.

Класс порождаемый при помощи `recordclass` имеет вид: 

```
from recordclass import memoryslots, itemgetset

class C(memoryslots, metaclass=recordclasstype):
    
    __attrs__ = ('attr_1',...,'attr_m')
    
    attr_1 = itemgetset(0)
    ...
    attr_m = itemgetset(m-1)
    
    def __new__(cls, attr_1, ..., attr_m):
        'Create new instance of {typename}({arg_list})'
        return memoryslots.__new__(cls, attr_1, ..., attr_m)        
```
и т.д. следуя схеме определения `namedtuple`.

Как результат `recordclass` занимает столько же памяти, сколько и `namedtuple`, поддерживает быстрый доступ по `__getitem__`/`__setitem__` и по имени аттрибута через протокол дескрипторов.

## Structclass

В дискуссии было справедливо замечено, что экземпляры классов со `__slots__` также поддерживают быстрый доступ к полям объекта и занимают меньше памяти, чем `tuple` и экземпляры классов, созданных при помощи функции-фабрики `recordclass`.  Это происходит, потому что экземпляры классов со `__slots__` не хранят количество элементов, как `tuple` и другие (`PyObjectVar`), но хранят количество элементов и список атрибутов в своем типе (`PyHeapTypeObject`).

Поэтому был создан специальный прототип класса из которого посредством специального метакласса `arrayclasstype` могут создаваться классы, экземпляры которых могут занимать в памяти столько же, сколько  экземпляры классов со `__slots__`, но при этом совсем не используют `__slots__`. На его основе функция-фабрика может создавать классы, экземпляры которых во все подобны экземплярам, созданным при помощи `recordclass`, но занимают в памяти меньше места.

The class generated by `recordclass` looks like:

``` python
from recordclass.arrayclass import dataobject, dataobjectgetset, structclasstype

class C(dataobject, metaclass=structclasstype):
    
    __attrs__ = ('attr_1',...,'attr_m')
    
    attr_1 = dataclassgetset(0)
    ...
    attr_m = dataclassgetset(m-1)
    
    def __new__(cls, attr_1, ..., attr_m):
        'Create new instance of {typename}({arg_list})'
        return ArrayClass.__new__(cls, attr_1, ..., attr_m)
```
etc. following the definition scheme of `recordclass`.

As a result, `recordclass2`-created objects takes up as much memory as `__slots__`-based instances and also have same functionality as `recordclass`-created instances.



In [1]:
import sys

class Record:
    __slots__ = ()

    def __init__(self, *args):
        assert len(self.__slots__) == len(args)
        for field, value in zip(self.__slots__, args):
            setattr(self, field, value)

    def __getitem__(self, index):
        return getattr(self, self.__slots__[index])

    def __len__(self):
        return len(self.__slots__)

    def __eq__(self, that):
        if not isinstance(that, type(self)):
            return NotImplemented
        return all(item == iota for item, iota in zip(self, that))

    def __repr__(self):
        args = ', '.join(repr(item) for item in self)
        return '%s(%s)' % (type(self).__name__, args)

    def __getstate__(self):
        return tuple(self)

    def __setstate__(self, state):
        self.__init__(*state)

from collections import namedtuple
from namedlist import namedlist
from recordclass import recordclass, structclass

NTTest = namedtuple('NTTest', 'a b c d e f')
NLTest = namedlist('NLTest', 'a b c d e f')
RCTest = recordclass('RCTest', 'a b c d e f')
RCTest2 = structclass('RCTest2', 'a b c d e f')

class RETest(Record):
    __slots__ = 'a', 'b', 'c', 'd', 'e', 'f'

nttest = NTTest(1, 2, 3, 4, 5, 6)
nltest = NLTest(1, 2, 3, 4, 5, 6)
rctest = RCTest(1, 2, 3, 4, 5, 6)
rctest2 = RCTest2(1, 2, 3, 4, 5, 6)
retest = RETest(1, 2, 3, 4, 5, 6)

print('Attribute Access')
print('namedtuple')
%time for i in range(1000000): nttest.c
print('namedlist')
%time for i in range(1000000): nltest.c
print('recordclass')
%time for i in range(1000000): rctest.c
print('structclass')
%time for i in range(1000000): rctest2.c
print('Record')
%time for i in range(1000000): retest.c

print('\nIndex Access')
print('namedtuple')
%time for i in range(1000000): nttest[3]
print('namedlist')
%time for i in range(1000000): nltest[3]
print('recordclass')
%time for i in range(1000000): rctest[3]
print('structclass')
%time for i in range(1000000): rctest2[3]
print('Record')
%time for i in range(1000000): retest[3]

print('\nObject Size:')
print('namedtuple')
print(sys.getsizeof(nttest))
print('namedlist')
print(sys.getsizeof(nltest))
print('recordclass')
print(sys.getsizeof(rctest))
print('structclass')
print(sys.getsizeof(rctest2))
print('Record')
print(sys.getsizeof(retest))

Attribute Access
namedtuple
CPU times: user 169 ms, sys: 1.04 ms, total: 170 ms
Wall time: 170 ms
namedlist
CPU times: user 126 ms, sys: 2.15 ms, total: 128 ms
Wall time: 127 ms
recordclass
CPU times: user 111 ms, sys: 2.16 ms, total: 113 ms
Wall time: 113 ms
structclass
CPU times: user 137 ms, sys: 2.29 ms, total: 140 ms
Wall time: 139 ms
Record
CPU times: user 113 ms, sys: 2.19 ms, total: 116 ms
Wall time: 115 ms

Index Access
namedtuple
CPU times: user 116 ms, sys: 1.3 ms, total: 117 ms
Wall time: 117 ms
namedlist
CPU times: user 832 ms, sys: 9.13 ms, total: 841 ms
Wall time: 848 ms
recordclass
CPU times: user 124 ms, sys: 3.01 ms, total: 127 ms
Wall time: 127 ms
structclass
CPU times: user 153 ms, sys: 4.46 ms, total: 157 ms
Wall time: 158 ms
Record
CPU times: user 427 ms, sys: 6.68 ms, total: 434 ms
Wall time: 437 ms

Object Size:
namedtuple
96
namedlist
88
recordclass
72
structclass
64
Record
88


In [2]:
rctest2[-1]

6

In [4]:
A1=structclass('A', 'a b c')
A2=structclass('A', 'd e f', gc=True)
A3=structclass('A', 'd e f')
a=A1(1,2,3)
a3=A3(1,2,3)
print(a)
print(a3)

A(a=1, b=2, c=3)
A(d=1, e=2, f=3)


In [5]:
from recordclass import join_classes

class B(A1):
    pass

class C(B):
    pass

D = join_classes('D', [A1, A2])

b=B(1,2,3)
c=C(1,2,3)
d=D(1,2,3,4,5,6)
b.fff = 100
c.ggg = 200

In [6]:
b.__dict__, c.__dict__

({'fff': 100}, {'ggg': 200})

In [7]:
print(sys.getsizeof(a3))
print(sys.getsizeof(a))
print(sys.getsizeof(b))
print(sys.getsizeof(c))
print(sys.getsizeof(d))

40
40
48
48
64


In [8]:
Point = structclass('Point', 'x y')

In [9]:
class Vector(Point):
    pass

In [10]:
p = Point(1,2)

try:
    p.__dict__
except AttributeError:
    print('*** There is no __dict__ ***')
try:
    p.__weakref__
except AttributeError:
    print('*** There is no __weakref__ ***')

print(p, 'sizeof:', sys.getsizeof(p))

*** There is no __dict__ ***
*** There is no __weakref__ ***
Point(x=1, y=2) sizeof: 32


In [11]:
v = Vector(1,2)
print(v, 'sizeof:', sys.getsizeof(v))

Vector(x=1, y=2) sizeof: 40


In [12]:
print('__dict__:', v.__dict__)

__dict__: {}
